# Imports

In [1]:
import numpy as np
import pandas as pd
from os import listdir
import os
import time
import gc
from src import *

In [2]:
directory = r'Dataset\\'
datasets = listdir(directory)

# Parameter Selection

In [9]:
results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','oob_acc'])

depth_cv = [3,5,10]
ntree_cv = [100]
term_cv = [True]
rep_num = 1
method_list = ['l','d','b']
distance_list = ['manhattan']
normal_list = [False]
batch_list = list(range(10))
batch_list2 = list(range(100))


for d in datasets:
	data_name = d
	#print(data_name)
	train,test,labels_train,labels_test,X_train,X_test = load_data_new(directory,data_name)
	#print(data_name+" is loaded!")
	results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','var','oob_acc'])
	param_time = time.time()
	
	for method in method_list:

		represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
		#print(data_name+" is prepared!")
		#print("CV starts")
		for distance_measure in distance_list:
			for rep in range(rep_num):
				#print(rep)
				for depth in depth_cv:
					for ntree in ntree_cv:
						for is_terminal in term_cv: 
							for n in [0]:
								first = True
								counter = 0
								for b in batch_list:
									represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
									represent_train['Id'] = train_id
									represent_train = represent_train.dropna()
									train_id = represent_train['Id'].values
									represent_train = represent_train.drop(columns = ['Id'])


									represent_test['Id'] = test_id
									represent_test = represent_test.dropna()
									test_id = represent_test['Id'].values
									represent_test = represent_test.drop(columns = ['Id'])
                                    
									trainbow,testbow = learn_representation_sparse_2(represent_train, None,labels_train,labels_test, train_id, test_id, depth, int(ntree/len(batch_list)),rep*1000+counter, is_terminal,normal=False)
									counter = counter +1                    
                                    
									if first is True:
										distTrain=pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)       
										first = False
									else:
										distTrain=distTrain+ pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)
									del trainbow
									del testbow
									gc.collect()
								np.fill_diagonal(distTrain, 1000000000000)
								nncl=KNeighborsClassifier(n_neighbors=1,metric='precomputed')
								nnfit=nncl.fit(distTrain,labels_train)
								predicted_train = nnfit.predict(distTrain)
								cv_accuracy = accuracy_score(labels_train,predicted_train)
								temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[False],'rep':[rep+1],'depth':[depth],'ntree':[ntree],'var':[n],'type':[is_terminal],'oob_acc':[cv_accuracy]})

								results = results.append(temp_results,sort=False)
                                
summary = results.groupby(['method','normalize','depth','var']).mean().reset_index()[results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'] == results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'].max()].reset_index(drop=True).sort_values(['method'],ascending=False).iloc[0]

method = summary['method']
n = summary['normalize']
depth = summary['depth']
var = summary['var']
print('Parameter selection is completed')
print(summary)

Parameter selection is completed
method              d
normalize       False
depth              10
var                 0
oob_acc      0.888889
Name: 0, dtype: object


# Testing

In [12]:
test_results = pd.DataFrame()
nof_test_replicate = 5
for rep in range(nof_test_replicate):
    train_time = time.time()
    first = True
    counter = 0
    for b in batch_list2:

        represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)

        represent_train['Id'] = train_id
        represent_train = represent_train.dropna()


        train_id = represent_train['Id'].values
        represent_train = represent_train.drop(columns = ['Id'])


        represent_test['Id'] = test_id
        represent_test = represent_test.dropna()

        test_id = represent_test['Id'].values
        represent_test = represent_test.drop(columns = ['Id'])

        trainbow,testbow = learn_representation_sparse_2(represent_train, represent_test,labels_train,labels_test, train_id, test_id, depth, int(500/len(batch_list2)),rep*1000+counter, is_terminal,normal=n)
        counter = counter +1

        trainbow_1 = trainbow
        testbow_1 = testbow   


        if first is True:
            distTrainTest=pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs=-1) 
            first = False
        else:
            distTrainTest = distTrainTest +pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs = -1) 
        del trainbow
        del testbow
        gc.collect()

    train_time = time.time()-train_time
    test_time = time.time()
    predicted_test = np.array(labels_train)[np.argmin(distTrainTest, axis=0)]
    test_accuracy=accuracy_score(labels_test,predicted_test)

    test_time = time.time()-test_time
    temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[400],'type':[is_terminal],'test_acc':[test_accuracy],'param_time':[np.round(param_time,1)],'train_time':[np.round(train_time,1)],'test_time':[np.round(test_time,1)]})
    test_results = test_results.append(temp_results,sort=False)
    print('Rep: ',rep,' Test Accuracy: ',test_accuracy)


Rep:  0  Test Accuracy:  0.8285714285714286
Rep:  1  Test Accuracy:  0.8171428571428572
Rep:  2  Test Accuracy:  0.8228571428571428
Rep:  3  Test Accuracy:  0.8171428571428572
Rep:  4  Test Accuracy:  0.8
